In [2]:
#Script to pull transactions data from LooksRare Subgraph

import requests
import json
import pandas as pd
from datetime import datetime

# Point to correct subgraph URL
url = 'https://gateway.thegraph.com/api/[api-key]/subgraphs/id/HJaSxvt1vH2nawRS53fGdxSrBbUZnYh52V4k8MjiPa7t'


# initialize full_data
df1_fulldata = pd.DataFrame()
df2_fulldata = pd.DataFrame()
df_data = pd.DataFrame()

for i in range(0,200) :
    # Set query (which fetches 200,000 records from LooksRare subgraph)
    query = '''query {
    transactions(orderBy: date, orderDirection: desc, first:1000, skip:%s){
        id
        date
        isTakerAsk
        collection{
          id
        }
        maker{
          id
        }
        taker{
          id
        }
        price
        }
    }
    ''' % (i*1000)


    # Make the request
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    
    # JSON adjustment
    json_data = json.loads(r.text)
    
    # extract JSON to convert to a dataframe
    df_data = json_data['data']['transactions']
    df1_fulldata = pd.DataFrame(df_data)
        
    df2_fulldata = pd.concat([df2_fulldata,df1_fulldata])
    

#Removing NULLS - Data Cleaning
df = df2_fulldata.dropna()

#Converting from Unix timestamp to datetime
df['date'] = pd.to_datetime(df['date'],unit='s')

#Removing extra data from columns : Cleaning
df['makerID'] = df['maker'].map(lambda x: str(x)[8:-2])
df['takerID'] = df['taker'].map(lambda x: str(x)[8:-2])
df['collectionID'] = df['collection'].map(lambda x: str(x)[8:-2])

#Dropping the collection, maker, taker columns
df = df.drop(['collection'], axis=1)
df = df.drop(['maker'], axis=1)
df = df.drop(['taker'], axis=1)

print(df)

                                                    id                date  \
0    0xec20b0e86e1b4ed93129e8b3949d402b6ca24056abb0... 2022-07-29 17:10:10   
1    0xc0f64bbe11aa0cddb5a5881c42051543816334111a03... 2022-07-29 17:08:22   
2    0xa9883c12d06ff725cdf91a9f4845b0ff82faeac019eb... 2022-07-29 17:05:04   
3    0x023e1eea544f220975815922bc1810582d3ba098a68b... 2022-07-29 17:03:23   
4    0xd10015a3d198cdabd2424f241cedd7e6151b9af0c840... 2022-07-29 16:57:24   
..                                                 ...                 ...   
995  0xfae66142ca25327bc313a0a656d4841a40ab635cb398... 2022-03-11 20:34:47   
996  0x40cd80fd84f201813854e2cac51d0b39d9e00be34769... 2022-03-11 20:34:41   
997  0xdb4e5f7be5c0c9bf8bae046f618a54f40845096dd089... 2022-03-11 20:33:28   
998  0xc82981f590583d66691c23fe45337ebe27c2732399df... 2022-03-11 20:32:49   
999  0x70c2a01289c4d2ba676d2b1c21ae4a3a16009db9a713... 2022-03-11 20:31:28   

     isTakerAsk                 price  \
0         False       

In [3]:
df['transaction_date'] = pd.to_datetime(df['date']).dt.date

#Saving an original copy of DF
df_copy = df.dropna()

In [5]:
df.head()

,id,date,isTakerAsk,price,makerID,takerID,collectionID,transaction_date
0,0xec20b0e86e1b4ed93129e8b3949d402b6ca24056abb0...,2022-07-29 17:10:10,False,0.013,0xba39ae23c8eba7f0e1a7c5a9ba0d807661a6c148,0x83c8f28c26bf6aaca652df1dbbe0e1b56f8baba2,0x4ceb477a068350bc652cabff127f87ad874b11f9,2022-07-29
1,0xc0f64bbe11aa0cddb5a5881c42051543816334111a03...,2022-07-29 17:08:22,False,0.515,0x10836d0c0666fe33c5540a6dfb6959e8e929ddcb,0x83c8f28c26bf6aaca652df1dbbe0e1b56f8baba2,0x33c6eec1723b12c46732f7ab41398de45641fa42,2022-07-29
2,0xa9883c12d06ff725cdf91a9f4845b0ff82faeac019eb...,2022-07-29 17:05:04,False,0.022,0xe0b9c94a4ec635e437f62f8bf505836d6f41c078,0xf538ac1769709c0600c45c9eaadcf2c7d813dd79,0xb6d460ac51b93bca63b694f099c4a8b3b1cf73b4,2022-07-29
3,0x023e1eea544f220975815922bc1810582d3ba098a68b...,2022-07-29 17:03:23,True,0.47,0xbb5d1c5455ebb40cbe4adc6766e6465dcf94e7cc,0x596736970d727752abc35e43faf1b60e4e8562da,0x33c6eec1723b12c46732f7ab41398de45641fa42,2022-07-29
4,0xd10015a3d198cdabd2424f241cedd7e6151b9af0c840...,2022-07-29 16:57:24,False,2.098000000000000256,0x562607a01a12e84a4abe025ac14ab1e36b76519f,0x570cf51ac81d5871d7fa72fe1f02eb71670dacfa,0xbd3531da5cf5857e7cfaa92426877b022e612cf8,2022-07-29


In [6]:
#Extracting data with USD prices for daily prices conversion from WETH to USD

url2 = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

# initialize full_data for Query2 of Prices dataset
df3_fulldata = pd.DataFrame()
df1_data = pd.DataFrame()

# Set query (which uses text input to specify USD data)
query = '''query {
  tokenDayDatas(where:{token:"0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}, orderBy: date, orderDirection: desc, first: 500){
    token{
      symbol
    }
    priceUSD
    date
  }
}'''

# Make the request
r = requests.post(url2, json={'query': query})
#print(r.status_code)
#print(r.text)
    
# JSON adjustment 
json_data = json.loads(r.text)
    
# extract JSON to convert to a dataframe
df1_data = json_data['data']['tokenDayDatas']
df3_fulldata = pd.DataFrame(df1_data)

#Removing NULLS - Data Cleaning
df2 = df3_fulldata.dropna()

#Converting from Unix timestamp to datetime
df2['date'] = pd.to_datetime(df2['date'],unit='s')

#Removing extra data from columns : Cleaning
df2['token'] = df2['token'].map(lambda x: str(x)[12:-2])

print(df2)

    token                             priceUSD       date
0    WETH  1682.032884728967219179899946553404 2022-07-29
1    WETH  1725.217963931443712095645397854755 2022-07-28
2    WETH  1637.454653970609061251480013280175 2022-07-27
3    WETH  1448.249969359778742099280863267861 2022-07-26
4    WETH  1438.442903296398813999381172877339 2022-07-25
..    ...                                  ...        ...
495  WETH   1782.08782062251704977974663309068 2021-03-21
496  WETH  1809.114106230393504908287830192406 2021-03-20
497  WETH  1807.604599743042580599887514821981 2021-03-19
498  WETH   1778.85098684165670662065331690169 2021-03-18
499  WETH  1823.507178500705186808021123525388 2021-03-17

[500 rows x 3 columns]


In [7]:
#To match the datatypes before joining the dataframes
df1 = df.copy()

df2['transaction_date'] = df2['date']
df1['transaction_date']=df1['transaction_date'].astype('datetime64')
df2['priceUSD'] = pd.to_numeric(df2['priceUSD'])
df1['price'] = pd.to_numeric(df1['price'])

print(df1.dtypes)
print(df2.dtypes)

id                          object
date                datetime64[ns]
isTakerAsk                    bool
price                      float64
makerID                     object
takerID                     object
collectionID                object
transaction_date    datetime64[ns]
dtype: object
token                       object
priceUSD                   float64
date                datetime64[ns]
transaction_date    datetime64[ns]
dtype: object


In [9]:
#Left join on the above 2 dataframes based on 'Date'
df_final = pd.merge(df1, df2[['priceUSD', 'transaction_date']], on='transaction_date', how='left')

#Calculating price in USD based on daily USD rate
df_final['TransactionPrice_USD'] = df_final['priceUSD'] * df_final['price']
df_final.sort_values(by=['date'], inplace=True, ascending=False)

print(df_final)

                                                       id                date  \
0       0xec20b0e86e1b4ed93129e8b3949d402b6ca24056abb0... 2022-07-29 17:10:10   
1       0xc0f64bbe11aa0cddb5a5881c42051543816334111a03... 2022-07-29 17:08:22   
2       0xa9883c12d06ff725cdf91a9f4845b0ff82faeac019eb... 2022-07-29 17:05:04   
3       0x023e1eea544f220975815922bc1810582d3ba098a68b... 2022-07-29 17:03:23   
4       0xd10015a3d198cdabd2424f241cedd7e6151b9af0c840... 2022-07-29 16:57:24   
...                                                   ...                 ...   
199995  0xfae66142ca25327bc313a0a656d4841a40ab635cb398... 2022-03-11 20:34:47   
199996  0x40cd80fd84f201813854e2cac51d0b39d9e00be34769... 2022-03-11 20:34:41   
199997  0xdb4e5f7be5c0c9bf8bae046f618a54f40845096dd089... 2022-03-11 20:33:28   
199998  0xc82981f590583d66691c23fe45337ebe27c2732399df... 2022-03-11 20:32:49   
199999  0x70c2a01289c4d2ba676d2b1c21ae4a3a16009db9a713... 2022-03-11 20:31:28   

        isTakerAsk  price  